In [ ]:
import os
import pickle
import numpy as np
import pandas as pd
from itertools import chain
from sentence import Sentence
from sentence import TagPrefix
from sentence import TagSurfix

In [2]:
class DataHandler(object):
    def __init__(self, rootDir=None, dict_path=None, train_data=None):
        self.rootDir = rootDir
        self.dict_path = dict_path
        self.train_data = train_data
        self.spiltChar = ['。', '!', '！', '？', '?']
        self.max_len = 200
        self.totalLine = 0
        self.longLine = 0
        self.totalChars = 0
        self.TAGPRE = TagPrefix.convert()
    
    def loadData(self):
        self.loadRawData()
        self.handlerRawData()
    
    def loadRawData(self):
        self.datas = list()
        self.labels = list()
        if self.rootDir:
            print(self.rootDir)
            for dirName, subdirList, fileList in os.walk(self.rootDir):
                # curDir = os.path.join(self.rootDir, dirName)
                for file in fileList:                   
                    if file.endswith(".txt"): 
                        curFile = os.path.join(dirName, file)
                        print("processing:%s" % (curFile))
                        with open(curFile, "r", encoding='utf-8') as fp:
                            for line in fp.readlines():
                                self.processLine(line)

            print("total:%d, long lines:%d, total chars:%d" % (self.totalLine, self.longLine, self.totalChars))
            print('Length of datas is %d' % len(self.datas))
            print('Example of datas: ', self.datas[0])
            print('Example of labels:', self.labels[0])

    def processLine(self, line):
        line = line.strip()  
        nn = len(line)
        seeLeftB = False  
        start = 0
        sentence = Sentence()  
        try:
            for i in range(nn):  
                if line[i] == ' ':
                    if not seeLeftB:
                        token = line[start:i]
                        if token.startswith('['):
                            token_ = ''
                            for j in [i.split('/') for i in token.split('[')[1].split(']')[0].split(' ')]:
                                token_ += j[0]
                            token_ = token_ + '/' + token.split('/')[-1]
                            self.processToken(token_, sentence, False)
                        else:
                            self.processToken(token, sentence, False)
                        start = i + 1
                elif line[i] == '[':
                    seeLeftB = True
                elif line[i] == ']':
                    seeLeftB = False
        # 此部分未与上面处理方式统一，（小概率事件）数据多元化，增加模型泛化能力。
            if start < nn:
                token = line[start:]
                if token.startswith('['):
                    tokenLen = len(token)
                    while tokenLen > 0 and token[tokenLen - 1] != ']':
                        tokenLen = tokenLen - 1
                    token = token[1:tokenLen - 1]
                    ss = token.split(' ')
                    ns = len(ss)
                    for i in range(ns - 1):
                        self.processToken(ss[i], sentence, False)
                    self.processToken(ss[-1], sentence, True)
                else:
                    self.processToken(token, sentence, True)
        except Exception as e:
            print('处理数据异常, 异常行为：' + line)
            print(e)

    def processToken(self, tokens, sentence, end):
        nn = len(tokens)
        while nn > 0 and tokens[nn - 1] != '/':
            nn = nn - 1

        token = tokens[:nn - 1].strip() 
        tagPre = tokens[nn:].strip()
        tagPre = self.TAGPRE.get(tagPre, TagPrefix.general.value) 
        if token not in self.spiltChar:
            sentence.addToken(token, tagPre)
        if token in self.spiltChar or end:
            if sentence.chars > self.max_len: 
                self.longLine += 1
            else:
                x = []
                y = []
                self.totalChars += sentence.chars
                sentence.generate_tr_line(x, y)

                if len(x) > 0 and len(x) == len(y):
                    self.datas.append(x)
                    self.labels.append(y)
                else:
                    print('处理一行数据异常, 异常行如下')
                    print(sentence.tokens)
            self.totalLine += 1
            sentence.clear()

    def handlerRawData(self):
        self.df_data = pd.DataFrame({'words': self.datas, 'tags': self.labels}, index=range(len(self.datas)))
        self.df_data['sentence_len'] = self.df_data['words'].apply(
            lambda words: len(words)) 

        all_words = list(chain(*self.df_data['words'].values))
        sr_allwords = pd.Series(all_words)
        sr_allwords = sr_allwords.value_counts() 

        set_words = sr_allwords.index
        set_ids = range(1, len(set_words) + 1) 
        tags = ['x']

        for _, memberPre in TagPrefix.__members__.items():
            for _, memberSuf in TagSurfix.__members__.items():
                if memberSuf is TagSurfix.S and memberPre is TagPrefix.general:
                    tags.append(memberPre.value + memberSuf.value)
                elif memberSuf != TagSurfix.S:
                    tags.append(memberPre.value + memberSuf.value)

        tags = list(set(tags))
        print(tags)

        tag_ids = range(len(tags))

        self.word2id = pd.Series(set_ids, index=set_words)
        self.id2word = pd.Series(set_words, index=set_ids)
        self.id2word[len(set_ids) + 1] = '<NEW>'
        self.word2id['<NEW>'] = len(set_ids) + 1

        self.tag2id = pd.Series(tag_ids, index=tags)
        self.id2tag = pd.Series(tags, index=tag_ids)

        self.df_data['X'] = self.df_data['words'].apply(self.X_padding)
        self.df_data['y'] = self.df_data['tags'].apply(self.y_padding)

        self.X = np.asarray(list(self.df_data['X'].values))
        self.y = np.asarray(list(self.df_data['y'].values))
        print('X.shape={}, y.shape={}'.format(self.X.shape, self.y.shape))
        print('Example of words: ', self.df_data['words'].values[0])
        print('Example of X: ', self.X[0])
        print('Example of tags: ', self.df_data['tags'].values[0])
        print('Example of y: ', self.y[0])

        with open(self.dict_path, 'wb') as outp:
            pickle.dump(self.word2id, outp)
            pickle.dump(self.id2word, outp)
            pickle.dump(self.tag2id, outp)
            pickle.dump(self.id2tag, outp)
        print('** Finished saving the dict.')
        
        with open(self.train_data, 'wb') as outp:
            pickle.dump(self.X, outp)
            pickle.dump(self.y, outp)
        print('** Finished saving the train data.')

    def X_padding(self, words):

        ids = list(self.word2id[words])
        if len(ids) >= self.max_len: 
            return ids[:self.max_len]
        ids.extend([0] * (self.max_len - len(ids))) 
        return ids

    def y_padding(self, tags):
        ids = list(self.tag2id[tags])
        if len(ids) >= self.max_len:  
            return ids[:self.max_len]
        ids.extend([0] * (self.max_len - len(ids))) 
        return ids

In [3]:
data = DataHandler(rootDir='../corpus', dict_path='../data/your_dict.pkl', train_data='../data/your_train_data.pkl')
data.loadData()

print(data.X)
print(type(data.X))
print(data.X.shape)

../corpus
processing:../corpus\c1002-23995935.txt
processing:../corpus\c1002-23996898.txt
total:27, long lines:0, total chars:1289
Length of datas is 27
Example of datas:  ['人', '民', '网', '1', '月', '1', '日', '讯', '据', '《', '纽', '约', '时', '报', '》', '报', '道', '，', '美', '国', '华', '尔', '街', '股', '市', '在', '2', '0', '1', '3', '年', '的', '最', '后', '一', '天', '继', '续', '上', '涨', '，', '和', '全', '球', '股', '市', '一', '样', '，', '都', '以', '最', '高', '纪', '录', '或', '接', '近', '最', '高', '纪', '录', '结', '束', '本', '年', '的', '交', '易']
Example of labels: ['b', 'm', 'e', 'Date_b', 'Date_m', 'Date_m', 'Date_e', 's', 's', 's', 'b', 'm', 'm', 'e', 's', 'b', 'e', 's', 'b', 'e', 'b', 'm', 'e', 'b', 'e', 's', 'Date_b', 'Date_m', 'Date_m', 'Date_m', 'Date_e', 's', 'b', 'e', 'b', 'e', 'b', 'e', 'b', 'e', 's', 's', 'b', 'm', 'm', 'e', 'b', 'e', 's', 's', 's', 'b', 'm', 'm', 'e', 's', 'b', 'e', 'b', 'm', 'm', 'e', 'b', 'e', 's', 's', 's', 'b', 'e']
['e', 'Date_m', 's', 'Date_e', 'm', 'Date_b', 'b', 'x']
X.shape=(27, 200